In [1]:
using DrWatson
@quickactivate "dare"

using ReinforcementLearning
using IntervalSets
using LinearAlgebra
using ControlSystems
using CUDA
using DataStructures
using DataFrames
using PlotlyJS

include(srcdir("nodeconstructor.jl"))
include(srcdir("data_hook.jl"))
include(srcdir("env.jl"))
include(srcdir("sin_policy.jl"))
include(srcdir("plotting.jl"))
include(srcdir("custom_control.jl"));

WebIO._IJuliaInit()

### Define CM Matrix

In [2]:
CM = [0.0   1.0    0.0    0.0    2.0    3.0    4.0    5.0   6.0   7.0
     -1.0   0.0    0.0    0.0    0.0    8.0    0.0    0.0   0.0   9.0
      0.0   0.0    0.0    0.0   10.0   11.0   12.0   13.0   0.0  14.0
      0.0   0.0    0.0    0.0   15.0   16.0   17.0    0.0  18.0  19.0
     -2.0   0.0  -10.0  -15.0    0.0   20.0    0.0   21.0  22.0   0.0
     -3.0  -8.0  -11.0  -16.0  -20.0    0.0   23.0   24.0  25.0  26.0
     -4.0   0.0  -12.0  -17.0    0.0  -23.0    0.0   27.0  28.0  29.0
     -5.0   0.0  -13.0    0.0  -21.0  -24.0  -27.0    0.0   0.0  30.0
     -6.0   0.0    0.0  -18.0  -22.0  -25.0  -28.0    0.0   0.0   0.0
     -7.0  -9.0  -14.0  -19.0    0.0  -26.0  -29.0  -30.0   0.0   0.0]



10×10 Matrix{Float64}:
  0.0   1.0    0.0    0.0    2.0    3.0    4.0    5.0   6.0   7.0
 -1.0   0.0    0.0    0.0    0.0    8.0    0.0    0.0   0.0   9.0
  0.0   0.0    0.0    0.0   10.0   11.0   12.0   13.0   0.0  14.0
  0.0   0.0    0.0    0.0   15.0   16.0   17.0    0.0  18.0  19.0
 -2.0   0.0  -10.0  -15.0    0.0   20.0    0.0   21.0  22.0   0.0
 -3.0  -8.0  -11.0  -16.0  -20.0    0.0   23.0   24.0  25.0  26.0
 -4.0   0.0  -12.0  -17.0    0.0  -23.0    0.0   27.0  28.0  29.0
 -5.0   0.0  -13.0    0.0  -21.0  -24.0  -27.0    0.0   0.0  30.0
 -6.0   0.0    0.0  -18.0  -22.0  -25.0  -28.0    0.0   0.0   0.0
 -7.0  -9.0  -14.0  -19.0    0.0  -26.0  -29.0  -30.0   0.0   0.0

### Create the environment and thus the Nodeconstructor with the CM Matrix

In [3]:
env = SimEnv(num_sources=5, num_loads=5,CM=CM)

INFO: v_dc = 350V will get applied to all actions
INFO: Generating standard norm_array from nodeconstructor


# SimEnv

## Traits

| Trait Type        |                  Value |
|:----------------- | ----------------------:|
| NumAgentStyle     |          SingleAgent() |
| DynamicStyle      |           Sequential() |
| InformationStyle  | ImperfectInformation() |
| ChanceStyle       |           Stochastic() |
| RewardStyle       |           StepReward() |
| UtilityStyle      |           GeneralSum() |
| ActionStyle       |     MinimalActionSet() |
| StateStyle        |     Observation{Any}() |
| DefaultStateStyle |     Observation{Any}() |

## Is Environment Terminated?

No

## State Space

`Space{Vector{ClosedInterval{Float64}}}(ClosedInterval{Float64}[-1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0])`

## Action Space

`Space{Vector{ClosedInterval{Float64}}}(ClosedInterval{Float64}[-1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0])`

## Current State

```
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
```


### Define Policy

In [4]:
policy = RandomPolicy(action_space(env))
#policy = sin_policy()

typename(RandomPolicy)
├─ action_space => typename(Space)
│  └─ s => 15-element Vector{ClosedInterval{Float64}}
└─ rng => typename(Random._GLOBAL_RNG)


### Create Hook:
#### 2 Options:
   - pass the states to be stored to "collect_state_ids"
       - need to know the exact Name of the states in the Nodeconstructor
       - only states at L and C 
   - hand in the ID(Number) of Sources, Loads and Cables which states shall be stored
       - computes Voltage and Current at all R, L and C

In [7]:
hook = DataHook(collect_state_ids=[], collect_sources=[1,2], collect_cables=[1,2,3],collect_loads=[1],collect_action_ids=["u_v1_a"], collect_state_paras=get_state_paras(env.nc))

DataHook(false, "episode_data/", nothing, nothing, Any[0.0035784495592666972, 6.6761697096123e-6, 0.0035784495592666972, 9.089816940920157e-6, 0.0008729980976043039, 2.4997377340748733e-5, 0.0008729980976043039, 2.712873533610117e-6, 0.0033445666015055955, 7.002992057736374e-6  …  0.00021104131309299551, 0.002442294566032497, 6.0720122788127355, 35.749, 64.41600725452808, 5.853, 11.797007509471223, 41.088, 33.24500696128682, 36.84700771121106], Any[], Any[], Any[], [1, 2], [1, 2, 3], [1], Any[], Any[], ["u_v1_a"], 0×0 DataFrame, 0×0 DataFrame, 1, false, Float64[], 0.0, false, nothing, -1.0e6, 0, nothing, false)

In [13]:
run(policy, env, StopAfterEpisode(1), hook)

Any["source1_i_R1_a", "source1_i_R_C_a", "source1_i_R2_a", "source1_i_R_C_cables_a", "source1_i_R1_b", "source1_i_R_C_b", "source1_i_R2_b", "source1_i_R_C_cables_b", "source1_i_R1_c", "source1_i_R_C_c", "source1_i_R2_c", "source1_i_R_C_cables_c", "source2_i_R1_a", "source2_i_R_C_a", "source2_i_R2_a", "source2_i_R_C_cables_a", "source2_i_R1_b", "source2_i_R_C_b", "source2_i_R2_b", "source2_i_R_C_cables_b", "source2_i_R1_c", "source2_i_R_C_c", "source2_i_R2_c", "source2_i_R_C_cables_c", "cable1_i_R_a", "cable1_i_R_b", "cable1_i_R_c", "cable2_i_R_a", "cable2_i_R_b", "cable2_i_R_c", "cable3_i_R_a", "cable3_i_R_b", "cable3_i_R_c", ("load1_i_R_a", "load1_i_C_a", "load1_i_C_cables_a"), ("load1_i_R_b", "load1_i_C_b", "load1_i_C_cables_b"), ("load1_i_R_c", "load1_i_C_c", "load1_i_C_cables_c")]

DataHook(false, "episode_data/", [-400.05223847612575 -279.4506345381942 … 0.0 0.0; 149786.4858887885 0.0 … 0.0 0.0; … ; 0.0 0.0 … -4.919184501806342e-7 0.0; 0.0 0.0 … 0.0 0.0], [279.4506345381942 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Any[0.0035784495592666972, 6.6761697096123e-6, 0.0035784495592666972, 9.089816940920157e-6, 0.0008729980976043039, 2.4997377340748733e-5, 0.0008729980976043039, 2.712873533610117e-6, 0.0033445666015055955, 7.002992057736374e-6  …  0.00021104131309299551, 0.002442294566032497, 6.0720122788127355, 35.749, 64.41600725452808, 5.853, 11.797007509471223, 41.088, 33.24500696128682, 36.84700771121106], Any[1.4313798237066788, 0.0001869327518691444, 1.4313798237066788, 0.0001869327518691444, 1.4313798237066788, 0.0001869327518691444, 1.4313798237066788, 0.0001869327518691444, 1.4313798237066788, 0.0001869327518691444  …  6.671786391172762, 2.903260450342027, 2.903260450342027, 2.903260450342027, 4.466961727687194, 4.466961727

## Labeling in Dataframe:

In [19]:
env.nc.parameters["source"][1]

Dict{Any, Any} with 11 entries:
  "L1"    => 0.00357845
  "C"     => 6.67617e-6
  "fltr"  => "LCL"
  "pwr"   => 10000.0
  "L2"    => 0.00357845
  "R_C"   => 0.000186933
  "v_rip" => 0.0144773
  "vdc"   => 720
  "R1"    => 1.43138
  "i_rip" => 0.12271
  "R2"    => 1.43138

In [31]:
env.nc.parameters["cable"][1]

Dict{Any, Any} with 3 entries:
  "C" => 3.69628e-6
  "L" => 0.00243955
  "R" => 6.67179

In [30]:
env.nc.parameters["load"][1]

Dict{Any, Any} with 4 entries:
  "C"         => 6.072
  "L"         => 35.749
  "R"         => 98296.3
  "impedance" => "RLC"

## Column Title in Dataframe:
### Sources: source(id) _ (i/u) _ (Component-name)_(Phase)
### Cables: cable(id) _ (i/u) _ (Component-Name)_(Phase)
### Loads: load(id) _ (i/u) _ (Component-Name)_(Phase)


### Examples:

In [35]:
state_name1= "source2_i_R1_a"
state_name2= "cable3_u_R_b"
state_name3= "load1_i_C_a"

hook.df[:, Symbol(state_name3)]

500-element Vector{Float64}:
   0.0
   0.9001158322194358
   6.383562610694312
   7.473887696326451
   3.6433613517686396
   0.027123288661029598
  -0.6489733248366256
  -0.5045380268786201
   0.04520628694126836
  -6.048049233970998
   ⋮
 -22.607132076881637
 -24.967277130189863
 -23.591038526337616
 -11.36259759878951
   3.985651996860473
  12.74252764712622
  17.31898853358597
  14.538573121522896
   4.085769829480749
  -7.664933770054584